# String Method Analysis Markov-State-Models
## Imports

In [ ]:
import os
import pickle
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import glob


logging.getLogger("stringmethod").setLevel(logging.ERROR)
sys.path.append("../string-method-gmxapi/")
import src.analysis as spc

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

## Load data

This notebook needs to run in the string simulation folder, this cell will get you there. You also set up a path for writing the figures.

In [ ]:
name_sim = "C2I_lb_v1/"
path_raw = f"/data/sperez/Projects/string_sims/data/raw/{name_sim}"
path_interim = f"/data/sperez/Projects/string_sims/data/interim/{name_sim}"
path_processed = f"/data/sperez/Projects/string_sims/data/processed/{name_sim}"
path_report = f"/data/sperez/Projects/string_sims/reports/figures/{name_sim}"
os.chdir(path_raw)
os.getcwd()

In [ ]:
with open("cv.pkl", "rb") as file:
    cvs, ndx_groups = pickle.load(file)

The `load_swarm_data` function will load the swarm data in the `cv_coordinates`. If you set `extract=True` it will read the data from the swarm files. If you have done this previously you can set `extract=False` so the function just reads `postprocessing/cv_coordinates.npy`. `first_iteration` can be used to exclude initial swarms as equilibration and `last_iteration` can be done to exclude some iterations for example if you want to estimate the FES convergence by comparing blocks of data.

In [ ]:
extract = True
if extract:
    cv_coordinates = spc.load_swarm_data(
        extract=extract, first_iteration=100, last_iteration=None
    )
    np.save(f"{path_interim}cv_coordinates.npy", cv_coordinates)
else:
    cv_coordinates = np.load(f"{path_interim}cv_coordinates.npy")

In [ ]:
files = spc.natural_sort(glob.glob("./strings/string[0-9]*txt"))
strings = np.array([np.loadtxt(file).T for file in files])

# MSM modelling of free energy surface

## Dimensionality reduction with TICA

The following cell computes the tica projection of the string cvs and discards the tics that have the lowest kinetic variance. This reduces the cvs space to a lower dimensional space that is adapted to the kinetic variance. You can use the drop keyword to drop certain cvs that are not well converged in the string simulation or that change very little from the beggining to the end of the string. The best case scenario is that `drop=[]` just works.

In [ ]:
tica = spc.cvs_to_tica(cv_coordinates, drop=[20, 21, 22, 23, 32, 33, 34, 35])

## Cluster

The next cell plots the "vamp score" of using `n_clustercenters` to make an MSM. You should find that at some point the vamp score saturates. Choose the minimum number of clusters that gives you the saturated vamp score as the value of k for the next steps. This might take a little while.

In [ ]:
n_clustercenters = [5, 10, 30, 50, 75, 100, 200, 500][::-1]
fig, ax, vamp_scores = spc.get_vamp_vs_k(n_clustercenters, tica, n_jobs=4)  # 6 min
np.save(f"{path_interim}vamp_scores.npy", vamp_scores)

If the calculation fails, there is something wrong with your MSM. Either you have too little transitions or there too many cvs in tica to have all the states well connected. Solutions:
+ Reduce the maximum number of clusters (drop 200 and 500) of `n_clustercenters` and see if you get a saturated curve.
+ Reduce the number of cvs that went into your TICA calculation.
+ Do more iterations of the string method.

## MSM Deeptime

Choose the number of clusters, `k`, for the clustering from the previous calculation. Also change n_proc to however many processors you can use.

In [ ]:
k = 100
clusters = spc.k_means_cluster(tica, k, stride=1, max_iter=500, n_jobs=4, seed=28101990)

In [ ]:
%%time
msm, weights = spc.get_msm(clusters, n_jobs=4)

# FES projection on IG vs SF

## CVs for projection

Make a `cv_proj` numpy array with shape (n_iteration * n_swarms_iterations, n_frames_per_iter, 2). n_frames_per_iter is usally 2 since you only record the value of the cvs at the begining and end of the swarm. The last dimesions are the cvs on which you would like to project your FES using the weights obtained from the msm. The FES is then the negative log of a *weighted* histogram of the projection cvs using the weights from the msm. The projection cvs can be anything that you can calculate for a structure, not necessarily the cvs of the string. In the example bellow it is the mean of two cvs.

In [ ]:
cv_proj = spc.cvs_to_SF_IG(cv_coordinates, [0, 1], [10, 11])
np.save(f"{path_interim}cv_proj.npy", cv_proj)

## Project FES

Do the projection and take log. You have to choose a bandwidth for the [KDE](https://en.wikipedia.org/wiki/Kernel_density_estimation) of the histogram. It should be big enough to reduce noise but not so big to remove features. If you give `None`

In [ ]:
%%time
bandwidth = 0.05
p_of_cv, extent, _ = spc.get_kde(cv_proj, weights, bandwidth)
F0 = -np.log(p_of_cv)
F = F0 - F0.min()
F[F > 40] = np.nan

Do the projection and take log. You have to choose a bandwidth for the [KDE](https://en.wikipedia.org/wiki/Kernel_density_estimation) of the histogram. It should be big enough to reduce noise but not so big to remove features. If you give `None`

In [ ]:
np.save(f"{path_processed}FES_SF_IG.npy", F)

## Plot FES

In [ ]:
fig, ax = spc.plot_2D_heatmap(
    F,
    extent,
    f_max=25,
    f_min=0,
    cbar_label="Free Energy (kT)",
    xlabel="SF (nm)",
    ylabel="IG (nm)",
)
ax.set_xlim([0.48, 1.0])
ax.set_ylim([1.1, 2.45])
fig.tight_layout()
fig.savefig(path_report + "FES.png")

## Bootstrap to get error

The problem with calculating errors in MD is that most statistical techniques for this rely on the data being uncorrelated. MD data is most of the time highly correlated due to the proximity in time and starting structure. Correlated data generates artificially low error estimates. 

For this reason we use blocking. In our case we will use blocking+bootstrapping. This is very well explained in this [very usefull video](https://www.youtube.com/watch?v=gHXXGYIgasE&t=1854s) by prof. Giovanni Bussi.

The uncertainty is calculated as half of the interval containing 95% of the probability of the distribution of histograms generated in the bootstraps.

This part is probably going to be slow! Maybe it will go over night. It is actually doing len(blocks) * n_boot msms! The good things is that once you have figured out for your system (and similar systems) what is a reasonable number of blocks then you can just do `blocks=[my_reasonable_number_blocks]`.

In [ ]:
%%time
import src.analysis as spc

blocks = [2, 4, 8, 16, 32]
n_blocks = len(blocks)
n_boot = 100
calculate = False
if calculate:
    errors, his, hdis = spc.get_error(
        cv_proj,
        clusters,
        extent,
        n_boot=n_boot,
        bandwidth=bandwidth,
        nbin=55,
        blocks=blocks,
        seed=28101990,
        n_jobs=4,
    )
    np.save(f"{path_processed}errors_{n_boot}_{n_blocks}.npy", errors)
else:
    errors = np.load(f"{path_processed}errors_{n_boot}_{n_blocks}.npy")

Choose the number of blocks that gives you a high error.

Note,`e_min` and `e_max` are choosen to remove the extremely high or low values of error that are generated due to poor sampling or high free energy. These regions of the "free error surface" are not what we care about and thus we remove it from the statistic and the visualization.

In [ ]:
fig, ax = plt.subplots(1, 1)
for error in glob.glob(path_processed + "errors_*.npy"):
    errors = np.load(error)
    e_max = 6
    e_min = 1.0e-03
    e = errors.copy()
    e[e > e_max] = np.nan
    e[e <= e_min] = np.nan
    label = f"n_boot={error.split('/')[-1].split('_')[1]}"
    mean = np.nanmean(e, axis=(1, 2))
    std_err = np.nanstd(e, axis=(1, 2)) / np.sqrt(e.shape[0])
    ax.plot(np.array(blocks), mean, marker="o", label=label)
    ax.fill_between(np.array(blocks), mean + std_err, mean - std_err, alpha=0.3)
ax.legend()
ax.set_xlabel("Number of blocks", size=15)
ax.set_ylabel("FES error (kT)", size=15)

From the previous plot you can see which is the adequate number of blocks that low but still gives you the plateauing (or highest) error.

In [ ]:
number_blocks = 8
n_boot = 100
n_blocks = len(blocks)
errors = np.load(f"{path_processed}errors_{n_boot}_{n_blocks}.npy")
f_max = 20
e_max = 6
e_min = 1.0e-03

e = errors[blocks.index(number_blocks)].copy()
e[e > e_max] = np.nan
e[e <= e_min] = np.nan

fig, ax = plt.subplots(1, 2, figsize=(10 * 2, 7), sharex=True, sharey=True)
_ = spc.plot_2D_heatmap(
    F,
    extent,
    f_max=25,
    f_min=0,
    cbar_label="Free Energy (kT)",
    xlabel="SF (nm)",
    ylabel="IG (nm)",
    fig=fig,
    ax=ax[0],
)
_ = spc.plot_2D_heatmap(
    e,
    extent,
    f_max=e_max,
    f_min=0,
    cbar_label="FES Uncertainty (kT)",
    xlabel="SF (nm)",
    cmap=plt.cm.viridis_r,
    fig=fig,
    ax=ax[1],
)
ax[1].set_title("Bootstrap Error (95%)")
fig.tight_layout()
fig.savefig(path_report + "FES_error.png")

# Path CVs

Define path as the average of the last av_last_n_it of the strings. Obtain a reasonable guess of lambda.

In [ ]:
n_strings = strings.shape[0]
av_last_n_it = 25
path = np.mean(strings[n_strings - av_last_n_it :, :, :], axis=0)
lam = spc.get_path_lambda(path)
print(f"Lambda value for path {lam:.2f}")

Let's see if the lambda gives a well behaved path cv. The progress variable (s) should be increasing in the path itself and the distance to path variable (z) be low and constant.

## Representation of Path CV

### Load data and calculate path cvs

In [ ]:
cv_of_path = []
for p in path.T:
    cv_of_path.append(spc.cvs_to_path(p, path=path, lam=lam))
cv_of_path = np.array(cv_of_path)

In [ ]:
cvs_path = []
for i in range(cv_coordinates.shape[0]):
    cvs_path.append([])
    for j in range(cv_coordinates.shape[1]):
        cvs_path[i].append(spc.cvs_to_path(cv_coordinates[i, j, :], path=path, lam=lam))
cvs_path = np.array(cvs_path)

### Path CV on final string

Check how the path cv is represented in the average final string

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10 * 2, 7))
ax[0].plot(cv_of_path[:, 0], marker="o")
ax[1].plot(cv_of_path[:, 1], marker="o")
ax[0].set_xlabel("bead number")
ax[0].set_ylabel("S")
ax[0].set_xlabel("bead number")
ax[0].set_ylabel("Z")
ax[0].set_title(f"Path-variable {lam = :.2f}")
ax[1].set_title(f"Path-variable {lam = :.2f}")

### Path CV projected on IG vs SF

Check how the path variables project onto the canonical inactivation 2CV FES

In [ ]:
bandwidth = None
density_cut_off = -0.00001
print(f"If {density_cut_off:= e} we have F={-np.log(density_cut_off):= e}")

In [ ]:
s_of_cv, extent = spc.project_property_on_cv_kde(
    cv_proj,
    weights=weights,
    proper=cvs_path[:, :, 0:1],
    bandwidth=bandwidth,
    density_cut_off=density_cut_off,
)
np.save(f"{path_processed}s_path_of_SF_IG.npy", s_of_cv)

In [ ]:
z_of_cv, extent = spc.project_property_on_cv_kde(
    cv_proj,
    weights=weights,
    proper=cvs_path[:, :, 1:2],
    bandwidth=bandwidth,
    density_cut_off=density_cut_off,
)
np.save(f"{path_processed}z_path_of_SF_IG.npy", z_of_cv)

In [ ]:
s_of_cv[s_of_cv > 2] = np.nan

In [ ]:
z_of_cv[z_of_cv > 2] = np.nan

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10 * 2, 7), sharex=True, sharey=True)
_ = spc.plot_2D_heatmap(
    s_of_cv,
    extent,
    cbar_label="s[path]",
    xlabel="SF (nm)",
    ylabel="IG (nm)",
    f_min=0,
    f_max=1,
    fig=fig,
    cmap=plt.cm.Spectral,
    ax=ax[0],
    n_colors=200,
)
ax[0].contour(F, levels=20, extent=extent, vmin=0, vmax=20, colors="k")
ax[0].grid(None)
_ = spc.plot_2D_heatmap(
    z_of_cv,
    extent,
    cbar_label="z[path]",
    xlabel="SF (nm)",
    cmap=plt.cm.magma,
    f_min=0,
    f_max=0.4,
    fig=fig,
    ax=ax[1],
    n_colors=200,
)
ax[1].contour(F, levels=20, extent=extent, vmin=0, vmax=20, colors="w")
ax[1].grid(None)
fig.tight_layout()
fig.savefig(path_report + "projection_path_cv.png")

## Calculate FES projected on path CV

In [ ]:
import src.analysis as spc

In [ ]:
s_path = cvs_path[:, :, 0:1]

In [ ]:
%%time
bandwidth = 0.25
nbins = 55
p_of_cv, extent, kernel = spc.get_kde(s_path, weights, bandwidth, nbins=nbins)
F0 = -np.log(p_of_cv)
F = F0 - F0.min()
F[F > 40] = np.nan
s = np.linspace(extent[0], extent[1], nbins)
np.save(f"{path_processed}FES_path.npy",np.vstack([s, F]))

In [ ]:
%%time
import src.analysis as spc

blocks = [2, 4, 8, 16, 32]
n_blocks = len(blocks)
n_boot = 100
calculate = False
if calculate:
    errors, his, hdis = spc.get_error(
        s_path,
        clusters,
        extent,
        n_boot=n_boot,
        bandwidth=bandwidth,
        nbin=nbins,
        blocks=blocks,
        seed=28101990,
        n_jobs=4,
    )
    np.save(f"{path_processed}path_errors_{n_boot}_{n_blocks}.npy", errors)
    np.save(f"{path_processed}path_hdis_{n_boot}_{n_blocks}.npy", hdis)
else:
    errors = np.load(f"{path_processed}path_errors_{n_boot}_{n_blocks}.npy")
    hdis = np.load(f"{path_processed}path_hdis_{n_boot}_{n_blocks}.npy")

Choose the number of blocks that gives you a high error.

Note,`e_min` and `e_max` are choosen to remove the extremely high or low values of error that are generated due to poor sampling or high free energy. These regions of the "free error surface" are not what we care about and thus we remove it from the statistic and the visualization.

In [ ]:
fig, ax = plt.subplots(1, 1)
for error in glob.glob(path_processed + "path_errors_*.npy"):
    errors = np.load(error)
    e_max = 6
    e_min = 1.0e-03
    e = errors.copy()
    e[e > e_max] = np.nan
    e[e <= e_min] = np.nan
    label = f"n_boot={error.split('/')[-1].split('_')[2]}"
    mean = np.nanmean(e, axis=1)
    std_err = np.nanstd(e, axis=1) / np.sqrt(e.shape[0])
    ax.plot(np.array(blocks), mean, marker="o", label=label)
    ax.fill_between(np.array(blocks), mean + std_err, mean - std_err, alpha=0.3)
ax.legend()
ax.set_xlabel("Number of blocks", size=15)
ax.set_ylabel("FES error (kT)", size=15)

In [ ]:
error_block = 16
n_boot = 50
n_blocks = len(blocks)
fig, ax = plt.subplots(1, 1)
hdis = np.load(f"{path_processed}path_hdis_{n_boot}_{n_blocks}.npy")
low = hdis[blocks.index(error_block), 0, :]
high = hdis[blocks.index(error_block), 1, :]
ax.plot(s, F, label=label)
ax.fill_between(s, high, low, alpha=0.3)
ax.legend()
ax.set_xlabel("s[path]", size=15)
ax.set_ylabel("F (kT)", size=15)
fig.savefig(path_report + "FES_path_cv.png")